In [2]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [15]:
# ailment, effect, negatives df with strain ID as index
df = pd.read_csv('../data/merged_pivot.csv', skiprows=[0]) # , index_col=['Unnamed: 0']

In [25]:
# strain csv with strain ID as index
strains_df = pd.read_csv('../data/strain.csv')

In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [19]:
df.head()

,Unnamed: 0,ammonia,apple,apricot,berry,blueberry,butter,cheese,chemical,chestnut,citrus,coffee,diesel,earthy,flowery,fruit,grape,grapefruit,honey,lavender,lemon,lime,mango,menthol,mint,minty,nutty,orange,peach,pear,pepper,pine,pineapple,plum,pungent,rose,sage,skunk,spicy/herbal,strawberry,sweet,tar,tea,tobacco,tree,tropical,vanilla,violet,woody,ammonia.1,apple.1,apricot.1,berry.1,blueberry.1,butter.1,cheese.1,chemical.1,chestnut.1,citrus.1,coffee.1,diesel.1,earthy.1,flowery.1,fruit.1,grape.1,grapefruit.1,honey.1,lavender.1,lemon.1,lime.1,mango.1,menthol.1,mint.1,minty.1,nutty.1,orange.1,peach.1,pear.1,pepper.1,pine.1,pineapple.1,plum.1,pungent.1,rose.1,sage.1,skunk.1,spicy/herbal.1,strawberry.1,sweet.1,tar.1,tea.1,tobacco.1,tree.1,tropical.1,vanilla.1,violet.1,woody.1,ammonia.2,apple.2,apricot.2,berry.2,blueberry.2,butter.2,cheese.2,chemical.2,chestnut.2,citrus.2,coffee.2,diesel.2,earthy.2,flowery.2,fruit.2,grape.2,grapefruit.2,honey.2,lavender.2,lemon.2,lime.2,mango.2,menthol.2,mint.2,minty.2,nutty.2,orange.2,peach.2,pear.2,pepper.2,pine.2,pineapple.2,plum.2,pungent.2,rose.2,sage.2,skunk.2,spicy/herbal.2,strawberry.2,sweet.2,tar.2,tea.2,tobacco.2,tree.2,tropical.2,vanilla.2,violet.2,woody.2,ammonia.3,apple.3,apricot.3,berry.3,blueberry.3,butter.3,cheese.3,chemical.3,chestnut.3,citrus.3,coffee.3,diesel.3,earthy.3,flowery.3,fruit.3,grape.3,grapefruit.3,honey.3,lavender.3,lemon.3,lime.3,mango.3,menthol.3,mint.3,minty.3,nutty.3,orange.3,peach.3,pear.3,pepper.3,pine.3,pineapple.3,plum.3,pungent.3,rose.3,sage.3,skunk.3,spicy/herbal.3,strawberry.3,sweet.3,tar.3,tea.3,tobacco.3,tree.3,tropical.3,vanilla.3,violet.3,woody.3,ammonia.4,apple.4,apricot.4,berry.4,blueberry.4,butter.4,cheese.4,chemical.4,chestnut.4,citrus.4,coffee.4,diesel.4,earthy.4,flowery.4,fruit.4,grape.4,grapefruit.4,honey.4,lavender.4,lemon.4,lime.4,mango.4,menthol.4,mint.4,minty.4,nutty.4,orange.4,peach.4,pear.4,pepper.4,pine.4,pineapple.4,plum.4,pungent.4,rose.4,sage.4,skunk.4,spicy/herbal.4,strawberry.4,sweet.4,tar.4,tea.4,tobacco.4,tree.4,tropical.4,vanilla.4,violet.4,woody.4,aroused,creative,energetic,euphoric,focused,giggly,happy,hungry,relaxed,sleepy,talkative,tingly,uplifted,anxious,dizzy,dry eyes,dry mouth,headache,paranoid,add/adhd,anorexia,anxiety,arthritis,asthma,bipolar disorder,cancer,cramps,crohns disease,depression,epilepsy,eye pressure,fatigue,fibromyalgia,gastrointestinal disorder,headaches,inflammation,insomnia,lack of appetite,migraines,muscle spasms,nausea,pain,ptsd,spasticity,stress
0,3534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,52.0,26.0,NaN,40.0,NaN,NaN,NaN,NaN,NaN,38.0,NaN,13.0,31.0,46.0,11.0,13.0,NaN,NaN,48.0,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,38.0,NaN,NaN,50.0
1,3535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [9]:
strains_df.head()

,name,type,rating
strain_id,,,
3534,100-og,hybrid,4.0
3535,1024,sativa,4.4
3536,13-dawgs,hybrid,4.2
3537,24k-gold,hybrid,4.6
3538,3-kings,hybrid,4.4


In [ ]:
flavors = df.loc[:,'strain_id':'woody']

In [11]:
flavors.head()

,strain_id,ammonia,apple,apricot,berry,blueberry,butter,cheese,chemical,chestnut,citrus,coffee,diesel,earthy,flowery,fruit,grape,grapefruit,honey,lavender,lemon,lime,mango,menthol,mint,minty,nutty,orange,peach,pear,pepper,pine,pineapple,plum,pungent,rose,sage,skunk,spicy/herbal,strawberry,sweet,tar,tea,tobacco,tree,tropical,vanilla,violet,woody
0,3534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,3537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
effects = df.loc[:, 'aroused':'uplifted']

In [13]:
effects.head()

,aroused,creative,energetic,euphoric,focused,giggly,happy,hungry,relaxed,sleepy,talkative,tingly,uplifted
0,NaN,42.0,NaN,52.0,26.0,NaN,40.0,NaN,NaN,NaN,NaN,NaN,38.0
1,NaN,31.0,37.0,NaN,NaN,NaN,62.0,NaN,50.0,NaN,NaN,NaN,43.0
2,NaN,24.0,NaN,41.0,NaN,NaN,57.0,NaN,59.0,NaN,NaN,NaN,43.0
3,NaN,33.0,NaN,48.0,NaN,NaN,62.0,NaN,66.0,NaN,NaN,NaN,48.0
4,NaN,NaN,NaN,50.0,NaN,NaN,73.0,NaN,90.0,46.0,NaN,NaN,46.0


In [14]:
negatives = df.loc[:, 'anxious':'paranoid']
negatives['strain_id'] = df['strain_id']

In [16]:
ailments = df.loc[:, 'add/adhd':'stress']
ailments['strain_id'] = df['strain_id']

In [19]:
ailments = ailments[['anxiety', 'depression', 'fatigue', 'headaches', 'lack of appetite', 'pain', 'stress', 'strain_id']]

In [48]:
# PICK CLASSES TO WORK WITH - reduce dimensionality as much as possible by most common 
negatives.head()

,anxious,dizzy,dry eyes,dry mouth,headache,paranoid,strain_id
0,0.0,13.0,31.0,46.0,11.0,13.0,3534
1,0.0,6.0,25.0,25.0,0.0,0.0,3535
2,2.0,6.0,10.0,21.0,2.0,0.0,3537
3,3.0,6.0,13.0,22.0,0.0,3.0,3538
4,3.0,5.0,9.0,21.0,0.0,0.0,3539


In [49]:
negatives.describe()

,anxious,dizzy,dry eyes,dry mouth,headache,paranoid,strain_id
count,1026.000000,1026.000000,1026.000000,1026.000000,1026.00000,1026.000000,1026.000000
mean,3.303119,6.068226,15.332359,27.308967,2.55653,4.449318,4417.733918
std,3.492533,4.504223,7.416111,9.080725,3.48169,4.497356,508.592163
min,0.000000,0.000000,0.000000,4.000000,0.00000,0.000000,3534.000000
25%,0.000000,3.000000,10.000000,21.000000,0.00000,0.000000,3973.250000
50%,3.000000,6.000000,15.000000,27.000000,1.00000,4.000000,4424.000000
75%,5.000000,8.000000,20.000000,33.000000,4.00000,7.000000,4850.750000
max,26.000000,42.000000,63.000000,70.000000,30.00000,34.000000,5298.000000


In [38]:
ailments.head()

,anxiety,depression,fatigue,headaches,lack of appetite,pain,stress,strain_id
0,48.0,25.0,NaN,NaN,NaN,38.0,50.0,3534
1,18.0,15.0,NaN,NaN,NaN,21.0,21.0,3535
2,23.0,25.0,NaN,NaN,NaN,18.0,28.0,3537
3,25.0,31.0,NaN,NaN,18.0,22.0,36.0,3538
4,28.0,32.0,NaN,NaN,NaN,32.0,42.0,3539


In [39]:
ailments.describe()

,anxiety,depression,fatigue,headaches,lack of appetite,pain,stress,strain_id
count,909.000000,969.000000,236.000000,88.000000,128.000000,967.000000,1016.000000,1026.000000
mean,28.915292,27.570691,17.199153,18.397727,21.382812,30.594623,37.807087,4417.733918
std,7.592490,7.142977,4.784567,6.195178,7.939854,10.040760,8.963208,508.592163
min,7.000000,7.000000,7.000000,8.000000,10.000000,7.000000,5.000000,3534.000000
25%,24.000000,23.000000,14.000000,15.000000,16.000000,23.000000,32.000000,3973.250000
50%,28.000000,27.000000,16.500000,17.500000,20.000000,30.000000,38.000000,4424.000000
75%,33.000000,31.000000,20.000000,20.250000,26.000000,37.000000,42.000000,4850.750000
max,66.000000,62.000000,34.000000,41.000000,57.000000,73.000000,106.000000,5298.000000


In [40]:
effects.head()

,aroused,creative,energetic,euphoric,focused,giggly,happy,hungry,relaxed,sleepy,talkative,tingly,uplifted
0,NaN,42.0,NaN,52.0,26.0,NaN,40.0,NaN,NaN,NaN,NaN,NaN,38.0
1,NaN,31.0,37.0,NaN,NaN,NaN,62.0,NaN,50.0,NaN,NaN,NaN,43.0
2,NaN,24.0,NaN,41.0,NaN,NaN,57.0,NaN,59.0,NaN,NaN,NaN,43.0
3,NaN,33.0,NaN,48.0,NaN,NaN,62.0,NaN,66.0,NaN,NaN,NaN,48.0
4,NaN,NaN,NaN,50.0,NaN,NaN,73.0,NaN,90.0,46.0,NaN,NaN,46.0


In [118]:
effects.describe()

,aroused,creative,energetic,euphoric,focused,giggly,happy,hungry,relaxed,sleepy,talkative,tingly,uplifted
count,9.000000,353.000000,289.000000,955.000000,164.000000,40.000000,1016.000000,210.000000,857.000000,365.000000,27.000000,23.000000,822.000000
mean,38.444444,39.773371,45.342561,48.673298,39.390244,36.100000,55.166339,34.695238,61.571762,42.969863,40.037037,33.434783,45.273723
std,7.584707,9.851435,11.166214,10.122006,11.101601,7.625194,10.840577,8.658457,14.152864,11.081698,8.285676,10.374033,10.922795
min,29.000000,17.000000,18.000000,15.000000,15.000000,22.000000,20.000000,20.000000,24.000000,17.000000,20.000000,7.000000,19.000000
25%,30.000000,33.000000,38.000000,43.000000,32.000000,30.750000,48.000000,28.250000,51.000000,34.000000,35.000000,28.500000,37.000000
50%,40.000000,39.000000,44.000000,49.000000,38.000000,36.500000,55.000000,33.500000,61.000000,42.000000,40.000000,31.000000,45.000000
75%,44.000000,44.000000,51.000000,55.000000,47.000000,42.000000,62.000000,39.000000,71.000000,50.000000,44.500000,38.000000,53.000000
max,50.000000,85.000000,85.000000,91.000000,75.000000,52.000000,112.000000,73.000000,115.000000,90.000000,56.000000,61.000000,85.000000


In [121]:
effects = effects[['creative', 'energetic', 'euphoric', 'focused', 'happy', 'hungry', 'relaxed', 'sleepy']]

In [147]:
effects.head()

,aroused,creative,energetic,euphoric,focused,giggly,happy,hungry,relaxed,sleepy,talkative,tingly,uplifted
0,NaN,42.0,NaN,52.0,26.0,NaN,40.0,NaN,NaN,NaN,NaN,NaN,38.0
1,NaN,31.0,37.0,NaN,NaN,NaN,62.0,NaN,50.0,NaN,NaN,NaN,43.0
2,NaN,24.0,NaN,41.0,NaN,NaN,57.0,NaN,59.0,NaN,NaN,NaN,43.0
3,NaN,33.0,NaN,48.0,NaN,NaN,62.0,NaN,66.0,NaN,NaN,NaN,48.0
4,NaN,NaN,NaN,50.0,NaN,NaN,73.0,NaN,90.0,46.0,NaN,NaN,46.0


In [124]:
# put all together.. ignore flavor for right now

In [20]:
narrowed = df[['Unnamed: 0','anxious', 'dizzy', 'dry eyes', 'dry mouth', 'headache', 'paranoid','creative', 'energetic', 'euphoric', 'focused', 'happy', 'hungry', 'relaxed', 'sleepy', 'anxiety', 'depression', 'fatigue', 'headaches', 'lack of appetite', 'pain', 'stress']]

In [17]:
negative_list = ['anxious', 'dizzy', 'dry eyes', 'dry mouth', 'headache', 'paranoid']
effect_list = ['creative', 'energetic', 'euphoric', 'focused', 'happy', 'hungry', 'relaxed', 'sleepy']
ailment_list = ['anxiety', 'depression', 'fatigue', 'headaches', 'lack of appetite', 'pain', 'stress']

In [21]:
narrowed.head()

,Unnamed: 0,anxious,dizzy,dry eyes,dry mouth,headache,paranoid,creative,energetic,euphoric,focused,happy,hungry,relaxed,sleepy,anxiety,depression,fatigue,headaches,lack of appetite,pain,stress
0,3534,NaN,13.0,31.0,46.0,11.0,13.0,42.0,NaN,52.0,26.0,40.0,NaN,NaN,NaN,48.0,25.0,NaN,NaN,NaN,38.0,50.0
1,3535,NaN,6.0,25.0,25.0,NaN,NaN,31.0,37.0,NaN,NaN,62.0,NaN,50.0,NaN,18.0,15.0,NaN,NaN,NaN,21.0,21.0
2,3537,2.0,6.0,10.0,21.0,2.0,NaN,24.0,NaN,41.0,NaN,57.0,NaN,59.0,NaN,23.0,25.0,NaN,NaN,NaN,18.0,28.0
3,3538,3.0,6.0,13.0,22.0,NaN,3.0,33.0,NaN,48.0,NaN,62.0,NaN,66.0,NaN,25.0,31.0,NaN,NaN,18.0,22.0,36.0
4,3539,3.0,5.0,9.0,21.0,NaN,NaN,NaN,NaN,50.0,NaN,73.0,NaN,90.0,46.0,28.0,32.0,NaN,NaN,NaN,32.0,42.0


In [13]:
# Vector for NN model.. fill in NaNs with zero for sparse matrix
nar_vect = narrowed.fillna(0)

In [14]:
nar_vect.head()

,anxious,dizzy,dry eyes,dry mouth,headache,paranoid,creative,energetic,euphoric,focused,happy,hungry,relaxed,sleepy,anxiety,depression,fatigue,headaches,lack of appetite,pain,stress
3534,0.0,13.0,31.0,46.0,11.0,13.0,42.0,0.0,52.0,26.0,40.0,0.0,0.0,0.0,48.0,25.0,0.0,0.0,0.0,38.0,50.0
3535,0.0,6.0,25.0,25.0,0.0,0.0,31.0,37.0,0.0,0.0,62.0,0.0,50.0,0.0,18.0,15.0,0.0,0.0,0.0,21.0,21.0
3537,2.0,6.0,10.0,21.0,2.0,0.0,24.0,0.0,41.0,0.0,57.0,0.0,59.0,0.0,23.0,25.0,0.0,0.0,0.0,18.0,28.0
3538,3.0,6.0,13.0,22.0,0.0,3.0,33.0,0.0,48.0,0.0,62.0,0.0,66.0,0.0,25.0,31.0,0.0,0.0,18.0,22.0,36.0
3539,3.0,5.0,9.0,21.0,0.0,0.0,0.0,0.0,50.0,0.0,73.0,0.0,90.0,46.0,28.0,32.0,0.0,0.0,0.0,32.0,42.0


In [9]:
nar_vect.columns

Index(['anxious', 'dizzy', 'dry eyes', 'dry mouth', 'headache', 'paranoid', 'creative', 'energetic', 'euphoric', 'focused', 'happy', 'hungry', 'relaxed', 'sleepy', 'anxiety', 'depression', 'fatigue', 'headaches', 'lack of appetite', 'pain', 'stress'], dtype='object')

In [159]:
# X to sparse matrix

nar_mat = csr_matrix(nar_vect.values)

In [160]:
type(nar_mat)

scipy.sparse.csr.csr_matrix

In [161]:
knn_baseline = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [162]:
knn_baseline.fit(nar_mat)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [163]:
distance, strains_index = knn_baseline.kneighbors([nar_vect.iloc[5]])

In [164]:
# second array is a list of all of the closest 
strains_index

array([[   5,  698,  393,  493,    4,  745,  688,  616,  256, 1023,    6,
        1008,  587,  727,  509,  360,  271,  889,   17,  297]])

In [166]:
list_strains = []
for strain in strains_index:
    for index in strain:
        list_strains.append(index)

In [167]:
list_strains

[5,
 698,
 393,
 493,
 4,
 745,
 688,
 616,
 256,
 1023,
 6,
 1008,
 587,
 727,
 509,
 360,
 271,
 889,
 17,
 297]

In [168]:
# we got predictions
strains_df.iloc[list_strains]

,name,type,rating
strain_id,,,
3539,303-og,indica,4.2
4232,green-hornet,hybrid,4.2
3927,cherry-limeade,hybrid,4.3
4027,death-star,indica,4.6
3538,3-kings,hybrid,4.4
4279,hawaiian-mayan-gold,sativa,5.0
4222,green-avenger,hybrid,5.0
4150,fruity-pebbles,hybrid,4.4
3790,blue-steel,indica,4.3


In [ ]:
# pickle model

In [172]:
import pickle

In [173]:
with open('./data/model_pickle', 'wb') as f:
    pickle.dump(knn_baseline, f)

In [176]:
# EXAMPLE SERIES to play with:
# I am going to need to turn the input into a series like this
# maybe have all of the columns out, and then assign a vector
# depending on what order they are passed in through
request_series = nar_vect.iloc[5]

In [178]:
distance, reccomendation = np.kneighbors([request_series])

In [179]:
reccomendation

array([[   5,  698,  393,  493,    4,  745,  688,  616,  256, 1023,    6,
        1008,  587,  727,  509,  360,  271,  889,   17,  297]])

In [180]:
list_strains = []
for strain in reccomendation:
    for index in strain:
        list_strains.append(index)

In [181]:
strains_df.iloc[list_strains]

,name,type,rating
strain_id,,,
3539,303-og,indica,4.2
4232,green-hornet,hybrid,4.2
3927,cherry-limeade,hybrid,4.3
4027,death-star,indica,4.6
3538,3-kings,hybrid,4.4
4279,hawaiian-mayan-gold,sativa,5.0
4222,green-avenger,hybrid,5.0
4150,fruity-pebbles,hybrid,4.4
3790,blue-steel,indica,4.3


In [183]:
effects = [
    "happy",
    "euphoric",
    "sleepy"
]
for index, effect in enumerate(effects):
    effects[index] = columns.index(effect)
effects

[10, 8, 13]